In [16]:
#load datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [37]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
def mae(ytrue, ypred): return mean_absolute_error(ytrue, ypred)
def mape(ytrue, ypred): return 100 * mean_absolute_percentage_error(ytrue, ypred)
def smape(ytrue, ypred): return 200 * np.mean(np.abs(ytrue - ypred) / (np.abs(ytrue) + np.abs(ypred)))
def rmse(ytrue, ypred): return math.sqrt(mean_squared_error(ytrue, ypred))
def rmae(ytrue, ypred, ynaive): return mean_absolute_error(ytrue, ypred) / mean_absolute_error(ytrue, ynaive)
def dae(ytrue, ypred):
    try:
        error = mae(ytrue.mean(axis=1), ypred.mean(axis=1))
    except:
        error = mae(ytrue, ypred)
    return error

In [111]:
os.chdir("c://Users//matis//OneDrive//Documents//ENSTA//3A//Capstone//project//tschora//EPFDAML")
from work.models.Feature import Naive
from work.models.Splitter import MySplitter
dataset = "2"

def load_naive_forecasts(dataset, country, labels):  
    naive_forecasts = {"train" : {}, "test" : {}, "validation": {}, "test_recalibrated" : {}}
    os.chdir("c://Users//matis//OneDrive//Documents//ENSTA//3A//Capstone//project//tschora//EPFDAML")

    if dataset in ("", "2", "3"): nval = 362
    else: nval = 365  
    spliter = MySplitter(nval, shuffle=False)
    model_wrapper = Naive("NAIVE", f"EPF{dataset}_{country}", labels) 
    # Need to re-split for taking the validation prices
    X, y = model_wrapper.load_train_dataset()
    ((Xtr, ytr), (Xv, yv)) = spliter(X, y)
    # Also computes the naive forecasts    
    naive_forecasts["validation"][country] = model_wrapper.predict(None, Xv) 
    naive_forecasts["train"][country] = model_wrapper.predict(None, Xtr)           
    Xt, yt = model_wrapper.load_test_dataset()
    naive_forecasts["test"][country] = model_wrapper.predict(None, Xt)
    naive_forecasts["test_recalibrated"][country] = model_wrapper.predict(None, Xt)  

    return naive_forecasts

In [124]:
country = "FRBE"
version = "2"


for country in ["FRBE", "DEBE"]:
    # print("country", country)
    os.chdir("c://Users//matis//OneDrive//Documents//ENSTA//3A//Capstone//project//tschora//EPFDAML//data")
    path_predict = f"datasets\EPF{version}_{country}\MLP_TSCHORA_EPF{version}_{country}_test_predictions.csv"
    df_predict = pd.read_csv(path_predict, sep=",", index_col=None)
    path_true = f"datasets/EPF{version}_{country}/test.csv"
    df_true = pd.read_csv(path_true, sep=",", index_col=None)

    if len(country)==4:
        countries = [country[:2], country[2:]]
    elif len(country)==6:
        countries = [country[:2], country[2:4], country[4:]]
    else:  countries = [country]

    cols_price = []
    for c in countries:
        cols_price += [c+"_price_"+str(i) for i in range(24)]
    
    naive = load_naive_forecasts(version, country, cols_price)


    # generalize metrics (on all countries)
    ytrue = df_true[cols_price].values.reshape(1,-1)
    ypred = df_predict.values.reshape(1,-1)
    ynaive = naive["test"][country].reshape(1,-1)
    # print(ytrue.shape, ypred.shape, ynaive.shape)
    mae_val,mape_val, smape_val, rmse_val, rmae_val, dae_val = mae(ytrue, ypred), mape(ytrue, ypred), smape(ytrue, ypred), rmse(ytrue, ypred), rmae(ytrue, ypred, ynaive), dae(ytrue, ypred)

    # single metrics (on single country)
    mae_list, mape_list, smape_list, rmse_list, rmae_list, dae_list = [], [], [], [], [], []
    for i, c in enumerate(countries):
        cols_c = [c+"_price_"+str(i) for i in range(24)]
        ytrue = df_true[cols_c].values.reshape(1,-1)
        ypred = df_predict.iloc[:, 24*i:24*(i+1)].values.reshape(1,-1)
        ynaive = naive["test"][country][:, 24*i:24*(i+1)].reshape(1,-1)
        mae_list.append(mae(ytrue, ypred))
        mape_list.append(mape(ytrue, ypred))
        smape_list.append(smape(ytrue, ypred))
        rmse_list.append(rmse(ytrue, ypred))
        rmae_list.append(rmae(ytrue, ypred, ynaive))
        dae_list.append(dae(ytrue, ypred))

    # print results in dataframe
    res_gen = pd.DataFrame({"MAE": mae_val, "MAPE": mape_val, "SMAPE": smape_val, "RMSE": rmse_val, "RMAE": rmae_val, "DAE": dae_val}, index=["General"])
    res_uni = pd.DataFrame({"MAE": mae_list, "MAPE": mape_list, "SMAPE": smape_list, "RMSE": rmse_list, "RMAE": rmae_list, "DAE": dae_list}, index=countries)
    res = pd.concat([res_gen, res_uni])
    print(f"Results for {country} - EPF{version} - MLP_TSCHORA_EPF{version}_{country}")
    print(res)
        





Results for FRBE - EPF2 - MLP_TSCHORA_EPF2_FRBE
              MAE       MAPE      SMAPE       RMSE      RMAE       DAE
General  5.721823  21.957466  15.008659  14.882179  0.736975  2.136997
FR       4.791156  15.025672  13.934142  12.382026  0.714893  1.602307
BE       6.652491  28.889260  16.083176  17.018929  0.753742  2.671687
Results for DEBE - EPF2 - MLP_TSCHORA_EPF2_DEBE
              MAE          MAPE      SMAPE       RMSE      RMAE       DAE
General  5.677762  1.082490e+14  18.201424  13.010289  0.705734  0.507860
DE       4.742462  2.164981e+14  20.642291   6.484919  0.652834  1.386799
BE       6.613062  2.897918e+01  15.760557  17.218626  0.749275  2.402519
